This code is © P. Sánchez and I. González, 2025, and it is made available under the GPL license enclosed with the software.

If you publish results using our algorithms, please acknowledge our work by citing the software directly as:

- Sánchez, P., González, I., Cortés, A., Carrillo, C., & Margalef, T. (2025). *Airborne Hyperspectral Data Processing* [Software]. Zenodo. [https://doi.org/10.5281/zenodo.14871861](https://doi.org/10.5281/zenodo.14871861)

This code was developed as part of the **SALUS (CPP2021-008762)** framework.

# Airborne Hyperspectral Data Processing

## Standardization of RGBN and LWIR Remote Sensing Data

### Overview

This code is designed to process and standardize two different remote sensing datasets: an RGBN dataset and a LWIR (Long-Wave Infrared) dataset. The primary objective is to align both datasets in terms of spatial resolution, coordinate reference system (CRS), and value range, ensuring they can be effectively analyzed together. Unlike previous implementations, this version does not assume specific input data but maintains the same standardization process for any given dataset.

### General Workflow

1. **Loading the Data**  
   - The code is structured to handle two input GeoTIFF files:  
     - An RGBN dataset containing Red, Green, Blue, and Near-Infrared bands.  
     - A LWIR dataset representing long-wave infrared data.  
   - These datasets may come from different sources and may have varying resolutions and CRS.  

2. **Bounding Box Intersection**  
   - To ensure spatial consistency, the code determines the overlapping area between both datasets.  
   - Only the intersecting region is processed, avoiding data misalignment issues.  

3. **Reprojecting and Resampling the LWIR Dataset**  
   - Since the RGBN and LWIR datasets might have different spatial resolutions or CRS, the LWIR dataset is reprojected and resampled to match the RGBN dataset.  
   - This step ensures that both datasets align properly, enabling accurate pixel-to-pixel comparisons.  

4. **Reading and Normalizing the Data**  
   - The spectral bands from both datasets are read as arrays.  
   - The RGBN bands (Red, Green, and Near-Infrared) are normalized to a standard range of 0 to 1.  
   - Similarly, the resampled LWIR data is adjusted to the same range.  

5. **Creating a Multi-Band GeoTIFF**  
   - The processed bands are saved into a single multi-band GeoTIFF file.  
   - This final output maintains the spatial alignment and standardization required for further analysis.

### Modifiable Variables for Input Data Processing  

Before running the standardization process, three key variables must be adjusted based on the specific input data:  

- **RGBN_tiff**: This variable should be set to the file path of the RGBN dataset, which contains Red, Green, Blue, and Near-Infrared bands.  
- **thermal_tiff**: This variable should be set to the file path of the LWIR (Long-Wave Infrared) dataset.  
- **max_band_value**: This value represents the maximum possible pixel value in the dataset. It must be determined from the particular metadata of the input data rather than assuming a fixed value.  

#### Standardization Considerations  

- The standardization process initially scales the data to a range between `0` and `1` using the theoretical maximum value.  
- If a more complex standardization method is necessary to retain critical information (e.g., histogram equalization or adaptive scaling), adjustments should be made at this stage.  
- Regardless of the chosen method, the final standardized values **must remain within the range `[0, 1]`** to ensure consistency and compatibility across datasets.  



In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from rasterio.mask import mask
from rasterio.warp import reproject, Resampling
from shapely.geometry import box
import os
from rasterio.features import shapes
import fiona
from shapely.geometry import shape, mapping
from shapely.ops import unary_union
from shapely.geometry import MultiPoint
from shapely import concave_hull

RGBN_tiff = "path_to_the-RGBN.tiff"
thermal_tiff = "path_to_the-LWIR.tiff"

max_band_value = 255 #This value needs to be taken from the metadata

# Load RGBN and LWIR datasets
with rasterio.open(RGBN_tiff) as src_rgbn, rasterio.open(thermal_tiff) as src_lwir:
    # Compute intersection of bounding boxes
    rgbn_bounds = box(*src_rgbn.bounds)
    lwir_bounds = box(*src_lwir.bounds)
    intersection = rgbn_bounds.intersection(lwir_bounds)

    if intersection.is_empty:
        raise ValueError("No overlapping area between RGBN and LWIR datasets.")

    # Read and reproject LWIR to match RGBN resolution and CRS
    lwir_resampled, _ = reproject(
        source=src_lwir.read(1),
        destination=np.empty((src_rgbn.height, src_rgbn.width), dtype=np.float32),
        src_transform=src_lwir.transform,
        src_crs=src_lwir.crs,
        dst_transform=src_rgbn.transform,
        dst_crs=src_rgbn.crs,
        resampling=Resampling.bilinear
    )

    # Read RGBN bands
    red = src_rgbn.read(1).astype(float) / 255  # Band 1 (Red)
    green = src_rgbn.read(2).astype(float) / 255  # Band 2 (Green)
    blue = src_rgbn.read(3).astype(float) / 255  # Band 3 (Blue)
    nir = src_rgbn.read(4).astype(float) / 255  # Band 4 (NIR)
    lwir = lwir_resampled / 255 # Use resampled LWIR

    # Save the profile of the GeoTIFF
    profile = src_rgbn.profile.copy()

output_tiff = "LWIR_normalized.tiff"
profile.update(
    dtype=rasterio.float32,
    count=1,  # 1 band
    driver="GTiff",
    compress='LZW',  # Compression for smaller file size
    tiled=True  # Enable tiling for compatibility with GIS tools
)

# Save the ratio as a new TIFF file
with rasterio.open(output_tiff, "w", **profile) as dst:
    dst.write(lwir, 1)  # Write the ratio as the first band

print(f"LWIR normalized saved to {output_tiff}")

## Creating and Saving the LNV.tiff

We want to generate a new raster file called **LNV.tiff**, which contains three bands:

1. **LWIR (Long-Wave Infrared)**: Represents thermal data, which is useful for detecting temperature variations in the image.
2. **NIR (Near-Infrared)**: Captures data in the near-infrared spectrum, which is critical for vegetation analysis and identifying heat signatures.
3. **Visible**: This is computed using the formula:

   $$
   \text{Visible} = 0.299 \cdot \text{Red} + 0.587 \cdot \text{Green} + 0.114 \cdot \text{Blue}
   $$

   The **Visible** band combines the red, green, and blue bands to create a representation of what is seen by the human eye, helping to enhance the visual context of the data.

### Why is LNV Useful?

The **LNV.tiff** file is particularly useful for a variety of remote sensing and environmental applications, including:

- **Fire Detection**: The combination of LWIR and NIR bands helps identify heat signatures, which are crucial for detecting fire hotspots. The **LWIR** band helps identify areas of high temperature (e.g., fire), while the **NIR** can be used to monitor the health of vegetation (fires often cause significant damage to plants).
  
- **Vegetation and Land Cover Monitoring**: The **NIR** band is highly sensitive to vegetation and is used in vegetation indices like NDVI. By combining this with the visible band, we can monitor land cover changes over time, including the impact of fires or other environmental phenomena.
  
- **Enhanced Visualization**: The **Visible** band, computed from the RGB bands, provides an intuitive representation of the data, allowing analysts to visually interpret the thermal and infrared information more easily.

### Code Explanation

1. **Visible Index Calculation**: The visible index is calculated by combining the RGB bands using weighted values that represent the relative contributions of red, green, and blue to the visible spectrum. This helps create a realistic image representation of the scene.

2. **Band Stacking**: The three bands (LWIR, NIR, and Visible) are stacked together into a single multi-band raster. This makes it easier to analyze and visualize the data as a cohesive unit.

3. **Saving the Output**: The resulting raster, **LNV.tiff**, is saved as a GeoTIFF with the three bands. The profile of the original RGBN image is copied and updated to accommodate the new bands and ensure compatibility with GIS tools. The file is compressed using **LZW** compression to reduce file size without losing data quality.

By creating and saving the **LNV.tiff**, we are preparing a valuable dataset for fire detection, environmental monitoring, and other geospatial analyses.


In [ ]:
output_tiff = "LNV.tiff" #Change if wanted

# Calculate the visible index using the formula
visible = (0.299 * red) + (0.587 * green) + (0.114 * blue)

# Stack the bands together: LWIR, NIR, and visible
indices = np.stack([lwir*255., nir*255., visible*255.], axis=0)

# Update the profile for the new output TIFF
profile.update(
    dtype=rasterio.float32,
    count=3,  # 3 bands: LWIR, NIR, visible
    driver="GTiff",
    compress='LZW',  # Compression for smaller file size
    tiled=True  # Enable tiling for compatibility with GIS tools
)

# Save the new TIFF with the 3 bands
with rasterio.open(output_tiff, "w", **profile) as dst:
    for i, index in enumerate(indices, start=1):
        dst.write(index, i)

print(f"LNV.tiff saved to {output_tiff}")

## Calculating the LWIR/NIR Ratio

The **LWIR/NIR ratio** is a critical index for detecting heat signatures and understanding thermal patterns in a scene. By comparing the **LWIR (Long-Wave Infrared)** and **NIR (Near-Infrared)** bands, this ratio can highlight areas of high temperature or significant thermal activity, which is especially useful for **fire detection**.

### Why is the LWIR/NIR Ratio Useful?

- **Fire Detection**: The **LWIR** band captures thermal information, which is essential for identifying heat sources, such as fires. The **NIR** band helps monitor vegetation and other land cover types. A higher ratio of LWIR to NIR may indicate elevated temperatures or fire-related phenomena, making this ratio a valuable tool for detecting hotspots or active fires in remote sensing data.
  
- **Thermal Anomalies**: This ratio can also be used to detect thermal anomalies in other scenarios, such as industrial areas or regions with significant temperature variation.

### Code Explanation

The code below computes the ratio between the **LWIR** and **NIR** bands, and saves the result as a **GeoTIFF** file. The ratio is calculated pixel-wise for each corresponding value in the LWIR and NIR bands.

In [ ]:
output_tiff = "LNIR.tiff" #Change if wanted

# Calculate the LWIR/NIR ratio (avoid division by zero by adding a small constant)
lnir_ratio = lwir*255 / (nir*255 + 1e-6)

# Update the profile for the new output TIFF
profile.update(
    dtype=rasterio.float32,
    count=1,  # 1 band
    driver="GTiff",
    compress='LZW',  # Compression for smaller file size
    tiled=True  # Enable tiling for compatibility with GIS tools
)

# Save the ratio as a new TIFF file
with rasterio.open(output_tiff, "w", **profile) as dst:
    dst.write(lnir_ratio, 1)  # Write the ratio as the first band

print(f"LWIR/NIR ratio saved to {output_tiff}")

## Importance of Fire Detection Indices

Fire-detection indices are critical tools used in remote sensing to identify and monitor active fires or fire-prone areas. These indices provide valuable insights into the thermal and spectral characteristics of fire-affected regions, allowing for better detection, analysis, and response to wildfire events. The indices we will compute here are particularly useful for distinguishing between fire and non-fire areas, and for assessing vegetation health and moisture content in the presence of fire. The indices we'll focus on include the **Burned Area Index (BAI)**, **Normalized Difference Vegetation Index (NDVI)**, and **Normalized Difference Water Index (NDWI)**.

Each of these indices relies on specific spectral bands, typically those related to visible light, infrared, and thermal wavelengths, which are sensitive to vegetation, water content, and temperature. By calculating and analyzing these indices, we can gain a better understanding of fire behavior, the extent of fire damage, and the recovery of affected vegetation.

### 1. Burned Area Index (BAI)

#### Formula:
$$
\text{BAI} = \frac{1}{\left( \left( \text{Red} - 0.1 \right)^2 + \left( \text{NIR} - 0.06 \right)^2 \right)}
$$


### Explanation:
The **Burned Area Index (BAI)** is designed to highlight areas that have been burned by fire. It is a vegetation index that uses the difference between the red (visible light) and near-infrared (NIR) bands to detect changes in vegetation after a fire. Healthy vegetation typically reflects a lot of infrared light, but when it is burned, it reflects less infrared light and more visible red light. The formula calculates the inverse of the squared difference between the red and NIR bands, making it highly sensitive to burned areas.

### 2. Normalized Difference Vegetation Index (NDVI)

#### Formula:
$$
\text{NDVI} = \frac{\text{NIR} - \text{Red}}{\text{NIR} + \text{Red}}
$$

#### Explanation:
The **Normalized Difference Vegetation Index (NDVI)** is one of the most commonly used indices for vegetation monitoring. It measures the difference between the near-infrared (NIR) and red bands to assess vegetation health. Healthy vegetation has a high NDVI value, while barren or burned areas have a low or negative value. NDVI is effective for monitoring vegetation greenness and biomass, and it can indicate the extent of damage from fire.

### 3. Normalized Difference Water Index (NDWI)

#### Formula:
$$
\text{NDWI} = \frac{\text{Green} - \text{NIR}}{\text{NIR} + \text{Green}}
$$

#### Explanation:
The **Normalized Difference Water Index (NDWI)** is used to monitor water content in vegetation. It uses the green and near-infrared (NIR) bands to distinguish between water and non-water surfaces. Water absorbs green light and reflects infrared light, so NDWI values are positive for water-rich areas and negative for dry areas. This index can help detect moisture changes caused by fire or drought.

### Saving Each Index Separately

The indices are calculated and saved separately as individual GeoTIFF files. Each index is saved as a single-band raster to preserve the individual information for further analysis or visualization.

For each index, the following steps are performed:
1. The index is calculated based on the respective formula.
2. A new GeoTIFF file is created for each index.
3. The index is written to its corresponding file using the `rasterio` library.
4. Each index is saved with compression and tiling enabled to optimize file size and compatibility with GIS tools.

After saving the indices, you can visualize them using a plotting function that displays each index with appropriate color maps.

In [ ]:
index_names = ["BAI", "NDVI", "NDWI"]

bai = 1 / ((red - 0.1) ** 2 + (nir - 0.06) ** 2 + 1e-6)
ndvi = (nir - red) / (nir + red + 1e-6)
ndwi = (green - nir) / (nir + green + 1e-6)

indices = np.stack([bai, ndvi, ndwi], axis=0)

# Save each index separately
for i, index in enumerate(indices):
    output_tiff = f"{index_names[i]}_index.tiff" #Change if wanted
    profile.update(
        dtype=rasterio.float32,
        count=1,  # Single-band output for each index
        driver="GTiff",
        compress='LZW',  # Reduce file size without losing precision
        tiled=True  # Might improve compatibility with GIS tools
    )
    with rasterio.open(output_tiff, "w", **profile) as dst:
        dst.write(index, 1)  # Write the index as the first band
    print(f"Index {index_names[i]} saved to {output_tiff}")

# Function to plot indices
def plot_index(index, title, cmap='hot', log_scale=False):
    plt.figure(figsize=(10, 6))
    plt.title(title)
    if log_scale:
        plt.imshow(np.log1p(index), cmap=cmap)
    else:
        plt.imshow(index, cmap=cmap)
    plt.colorbar(label=title)
    plt.show()

# Plot results
for i, index in enumerate(indices):
    plot_index(index, index_names[i], log_scale=(index_names[i] in ["BAI"]))


## Extracting Raw Hotspots from LWIR Data

In this section, we focus solely on extracting the raw hotspots from the **LWIR (Long-Wave Infrared)** data. This is done by applying a threshold based on a chosen percentile to identify the most intense heat signatures, which represent potential hotspots.

### Key Parameter:
- **Threshold Percentile (`threshold_percentile`)**: The percentile value is used to set the threshold above which the pixels are considered part of a hotspot. A common starting point is **98%**, but this can be adjusted depending on the data quality and fire intensity.

### Adapting the Threshold Percentile:
The value of `threshold_percentile` determines how sensitive the hotspot extraction process is to changes in temperature. Here's how to adapt the threshold:

1. **High Percentile (e.g., 98-99%)**:
   - **Effect**: Fewer, but more intense hotspots are detected. This is useful if you're interested in isolating the most significant fire activity, such as large or intense fire sources.
   - **When to Use**: This is ideal when the fire is very intense, and you're looking for the highest heat values in the scene.

2. **Lower Percentile (e.g., 95-98%)**:
   - **Effect**: More, but less intense hotspots are detected. This is helpful for detecting broader fire activity, including smoldering areas or lower-intensity fires.
   - **When to Use**: This is useful for broader analyses where you need to identify the full extent of a fire, including areas of lesser intensity.

3. **Dynamic Adjustment**:
   - It's often helpful to visually inspect the hotspots after applying different threshold values. If the extracted hotspots seem too sparse or too dense, adjust the percentile value accordingly.
   - For instance, if the hotspots are too numerous and you're interested in the main fire areas, try raising the threshold percentile. If you're missing smaller heat sources, lower the percentile.

By adjusting the `threshold_percentile`, you can fine-tune the hotspot extraction to suit the specific characteristics of the data and the fire's behavior.

### Methodology for Raw Hotspot Extraction:
1. **Extract Hotspots**: Using the `threshold_percentile`, we identify the pixels in the LWIR image that exceed this threshold, representing the hottest areas.
2. **Save the Raw Hotspot Data**: We save the raw hotspot data without any filtering based on size. These hotspots are saved as geometries in a `.shp` file for further analysis.

In [ ]:
LWIR_tiff = "LWIR_normalized.tiff"

# Threshold for detection (high percentiles to highlight fires)
threshold_percentile = 98

if os.path.exists(LWIR_tiff):  # Check if the LWIR file exists
    with rasterio.open(LWIR_tiff) as src:
        transform = src.transform
        crs = src.crs
        band = src.read(1)

        threshold = np.percentile(band, threshold_percentile)
        binary = (band >= threshold).astype(np.uint8)

        # Extract geometries
        geometries = [shape(geom) for geom, value in shapes(binary, transform=transform) if value == 1]

        if geometries:
            # Save the unprocessed hotspots
            output_filename_raw_LWIR = "hotspot_LWIR_raw.shp"

            # Create schema and save the raw hotspots
            schema = {'geometry': 'Polygon', 'properties': {'area': 'float'}}
            with fiona.open(output_filename_raw_LWIR, 'w', driver='ESRI Shapefile', crs=crs, schema=schema) as shp:
                for geom in geometries:
                    shp.write({'geometry': mapping(geom), 'properties': {'area': geom.area}})

            print(f"Raw LWIR hotspots saved to {output_filename_raw_LWIR}")
        else:
            print(f"No hotspots detected using LWIR.")


## Filtering LWIR Hotspots by Area

After the hotspots are identified, the next step is to filter them based on their area. This filtering is essential because not all detected hotspots are relevant; some may be noise or artifacts, especially in scenarios where smoke affects the data. To filter out such small, irrelevant hotspots, we calculate the area of each hotspot and apply a threshold. Only hotspots that have an area greater than or equal to the threshold are retained.


### How the Threshold Works

The area threshold is calculated as a percentage of the area of the largest detected hotspot. For example, if the largest hotspot in the data has an area of 1000 pixels, and we set the threshold to 1%, the minimum acceptable area for a hotspot will be 10 pixels. Hotspots smaller than this will be discarded.

This threshold is adjustable and must be defined by the user based on the quality of the raw hotspots:

- **If there is a lot of smoke**: Smoke can cause isolated, small hotspots that are likely noise. In such cases, the user should set a higher threshold (like `0.5`) to remove these small, isolated hotspots. This ensures that only larger, more likely fire hotspots are kept.
  
- **If the hotspots are clustered**: In situations where the hotspots are well-clustered and there is little smoke interference, a lower threshold (or even zero) can be used, as the small hotspots in the cluster are likely part of the same fire. In this case, there is no need to remove small hotspots.

The `threshold_area_LWIR` is a parameter that needs to be adjusted based on how the raw hotspots appear in your data. If the raw hotspots are highly dispersed due to smoke, increase the threshold. If they are more tightly clustered, decrease the threshold, or set it to zero to retain even the smallest hotspots.


## Error Handling

If, after applying the area threshold, all hotspots are discarded because their area was too small, you may encounter an error message like:

> "All fires detected using LWIR are too small"

This indicates that the `threshold_area_LWIR` is too high, and you will need to lower it to retain valid hotspots. This can happen especially if the raw hotspots were very small or if the fire is dispersed, resulting in many small hotspots that are still relevant.


In [ ]:
# Area threshold adjustment
threshold_area_LWIR = 0.01 #Change accordingly

hotspot_lwir = []

# Open the raw LWIR shapefile
with fiona.open(output_filename_raw_LWIR, 'r') as src:
    crs = src.crs
    geometries = [shape(feature['geometry']) for feature in src]

    # Calculate areas of the geometries
    areas = [geom.area for geom in geometries]

    if not areas:
        print(f"No fires detected using LWIR.")
    else:
        # Calculate the area threshold based on the largest geometry
        max_area = max(areas)
        area_threshold = max_area * threshold_area_LWIR

        # Filter geometries based on the area threshold
        filtered_geometries = [geom for geom in geometries if geom.area >= area_threshold]

        if not filtered_geometries:
            print(f"All fires detected using LWIR are too small.")
        else:
            # Merge filtered geometries into a single geometry
            merged_geometry = unary_union(filtered_geometries)
            hotspot_lwir.append(merged_geometry)

            # Define the output file path for the filtered hotspots
            output_filename_LWIR = "hotspot_LWIR.shp"

            # Save filtered hotspots to a new shapefile
            if hotspot_lwir:
                schema = {'geometry': 'Polygon', 'properties': {'area': 'float'}}
                combined_LWIR_geometry = unary_union(hotspot_lwir)
                with fiona.open(output_filename_LWIR, 'w', driver='ESRI Shapefile', crs=crs, schema=schema) as shp:
                    if combined_LWIR_geometry.geom_type == 'Polygon':
                        shp.write({'geometry': mapping(combined_LWIR_geometry), 'properties': {'area': combined_LWIR_geometry.area}})
                    elif combined_LWIR_geometry.geom_type == 'MultiPolygon':
                        for geom in combined_LWIR_geometry.geoms:
                            shp.write({'geometry': mapping(geom), 'properties': {'area': geom.area}})

                print(f"LWIR hotspots (filtered) saved to {output_filename_LWIR}")

## Extracting Raw Burned Areas from BAI Data

The extraction process for **BAI (Burned Area Index)** data differs from LWIR in that it focuses on detecting **burned areas** rather than **hotspots**. The threshold used in BAI is based on **fire extension** (i.e., the spread of the burn) rather than **fire intensity**.

### Key Considerations for BAI Data:

1. **Focus on Burned Area Extension**: Unlike LWIR, which identifies heat sources or hotspots related to active fire, **BAI identifies the extent of burned areas**. This means the threshold for BAI extraction does not depend on the intensity of the fire but on the spread of the fire, or how much area has already been burned.

2. **Threshold Percentile for BAI**: The **threshold_percentile_BAI** is used to define the extent of the burned area based on the changes in reflectance.
   - Higher percentiles (e.g., 98-99%) capture smaller, more concrete burned areas.
   - Lower percentiles (e.g., 95%) allow for the detection of larger burned areas, giving a less detailed map of the fire's spread.

### Why This Matters:

Since **BAI focuses on burned area extension**, the threshold is used to define the **area** affected by the fire. This means the threshold value should be adjusted based on the extension of the already burned area rather than fire intensity.


In [ ]:
BAI_tiff = "BAI_index.tiff"

# Threshold for detection
threshold_percentile = 98

if os.path.exists(BAI_tiff):  # Check if the BAI file exists
    with rasterio.open(BAI_tiff) as src:
        transform = src.transform
        crs = src.crs
        band = src.read(1)

        threshold = np.percentile(band, threshold_percentile)
        binary = (band >= threshold).astype(np.uint8)

        # Extract geometries
        geometries = [shape(geom) for geom, value in shapes(binary, transform=transform) if value == 1]

        if geometries:
            # Save the unprocessed burned areas
            output_filename_raw_BAI = "burnedarea_BAI_raw.shp"

            # Create schema and save the raw burned areas
            schema = {'geometry': 'Polygon', 'properties': {'area': 'float'}}
            with fiona.open(output_filename_raw_BAI, 'w', driver='ESRI Shapefile', crs=crs, schema=schema) as shp:
                for geom in geometries:
                    shp.write({'geometry': mapping(geom), 'properties': {'area': geom.area}})

            print(f"Raw BAI burned areas saved to {output_filename_raw_BAI}")
        else:
            print(f"No burned areas detected using BAI.")


## Filtering Burned Areas from BAI Data

After extracting the raw burned areas from **BAI (Burned Area Index)** data, it is important to filter out small, irrelevant areas that may be caused by **smoke interference**. Smoke can lead to false positives, resulting in the detection of small burned areas that are not representative of the actual fire's extent.

### How the Filtering Threshold Works:

The threshold for filtering is set based on the **size of the burned areas** and is typically defined by a percentage of the largest detected burned area. Burned areas smaller than this threshold are discarded as they are often noise caused by smoke or other environmental factors.

1. **Higher Threshold for Smoky Areas**: In regions with significant smoke, BAI data may contain small, isolated burned areas caused by smoke, not actual fire. In such cases, it is advisable to set a higher threshold (`0.5`) to filter out these small, scattered burned areas. A higher threshold will help ensure that only larger, more contiguous burned areas are retained, reducing the impact of smoke on the data.

2. **Lower Threshold for Cleaner Data**: In areas where smoke interference is minimal and the burned areas are well-clustered, a lower threshold (`0.05`) can be used. This allows for capturing more details of the fire's spread, including smaller burned areas that are still relevant.

### Why This Matters More using BAI than LWIR:

Because BAI is **much more sensitive to smoke**, it may detect small, irrelevant burned areas that are not indicative of the actual fire's extent. Filtering based on area helps remove these false positives, allowing for a clearer representation of the fire's true extent. The **threshold for filtering** should therefore be adjusted based on the level of smoke interference. If there is significant smoke, a higher threshold is necessary to remove small, isolated burned areas caused by smoke. Conversely, if the fire's spread is well-defined with minimal smoke, a lower threshold can be applied to retain more of the burned area data.


In [ ]:
# Area threshold adjustment
threshold_area_BAI = 0.01 #Change accordingly

burnedarea_BAI = []

# Open the raw BAI shapefile
with fiona.open(output_filename_raw_BAI, 'r') as src:
    crs = src.crs
    geometries = [shape(feature['geometry']) for feature in src]

    # Calculate areas of the geometries
    areas = [geom.area for geom in geometries]

    if not areas:
        print(f"No burned areas detected using BAI.")
    else:
        # Calculate the area threshold based on the largest geometry
        max_area = max(areas)
        area_threshold = max_area * threshold_area_BAI

        # Filter geometries based on the area threshold
        filtered_geometries = [geom for geom in geometries if geom.area >= area_threshold]

        if not filtered_geometries:
            print(f"All burned areas detected using BAI are too small.")
        else:
            # Merge filtered geometries into a single geometry
            merged_geometry = unary_union(filtered_geometries)
            burnedarea_BAI.append(merged_geometry)

            # Define the output file path for the filtered burned area
            output_filename_bai = "burnedarea_BAI.shp"

            # Save filtered burned areas to a new shapefile
            if burnedarea_BAI:
                schema = {'geometry': 'Polygon', 'properties': {'area': 'float'}}
                combined_bai_geometry = unary_union(burnedarea_BAI)
                with fiona.open(output_filename_bai, 'w', driver='ESRI Shapefile', crs=crs, schema=schema) as shp:
                    if combined_bai_geometry.geom_type == 'Polygon':
                        shp.write({'geometry': mapping(combined_bai_geometry), 'properties': {'area': combined_bai_geometry.area}})
                    elif combined_bai_geometry.geom_type == 'MultiPolygon':
                        for geom in combined_bai_geometry.geoms:
                            shp.write({'geometry': mapping(geom), 'properties': {'area': geom.area}})

                print(f"BAI burned areas (filtered) saved to {output_filename_bai}")

## Combining BAI Burned Areas and LWIR Hotspots to Determine Fire Extension

To estimate the **current extension of a fire** at a specific timestamp, it's essential to combine both **BAI Burned areas** and **LWIR hotspots**. Each dataset provides different but complementary insights into the fire's behavior, allowing for a more accurate understanding of its spatial spread.

- **BAI Burned Areas**: The BAI index indicates areas that have already burned. It represents the **fire's footprint**, showing the extent of the area affected by fire. This is crucial for understanding the overall area of destruction, even if the fire is no longer actively burning.

- **LWIR Hotspots**: On the other hand, LWIR hotspots capture areas with high thermal signatures, often associated with ongoing fire activity. These hotspots can indicate **active fire** and help identify the regions where the fire is currently spreading or intensifying.

### Why Combine Both Datasets?

Combining these two datasets is important for creating a comprehensive view of the fire's **spatial extension**. By merging BAI burned areas with LWIR hotspots, we can achieve the following:

1. **Capturing Ongoing and Past Fire Activity**: BAI shows where fire has already occurred, while LWIR hotspots identify where the fire is still active. By combining both, we capture the **total fire extension**, both in terms of already burned areas and areas with ongoing heat sources. This is especially useful for monitoring fire progression over time.

2. **More Accurate Fire Mapping**: Relying on only one of these datasets could lead to incomplete or misleading maps. For example, using only BAI might ignore the ongoing spread of the fire, while using only hotspots might miss areas that have already been burned but still have residual heat. The combination of both gives a clearer and more accurate picture.

3. **Improved Fire Management and Response**: For fire management, understanding the current and past extension of the fire is critical for resource allocation and decision-making. By combining BAI and LWIR hotspots, we can more effectively monitor fire dynamics and prioritize areas for intervention.

In [ ]:
if hotspot_lwir and burnedarea_BAI:
    all_fire = burnedarea_BAI + hotspot_lwir
    combined_geometry = unary_union(all_fire)

    output_filename_combined = "combined_fire.shp"

    if combined_geometry and not combined_geometry.is_empty:
        schema = {'geometry': 'Polygon', 'properties': {'area': 'float'}}
        with fiona.open(output_filename_combined, 'w', driver='ESRI Shapefile', crs=crs, schema=schema) as shp:
            if combined_geometry.geom_type == 'Polygon':
                shp.write({'geometry': mapping(combined_geometry), 'properties': {'area': combined_geometry.area}})
            elif combined_geometry.geom_type == 'MultiPolygon':
                for geom in combined_geometry.geoms:
                    shp.write({'geometry': mapping(geom), 'properties': {'area': geom.area}})

        print(f"Combined shapefile saved to {output_filename_combined}")
    else:
        print(f"No significant fires information.")

## Create a Concave Hull for Fire Extension

Once the **BAI burned areas** and **LWIR hotspots** have been combined, the next step is to derive the **current fire extension** approximately. This is done by generating a **concave hull** around the points obtained from both datasets.

A **concave hull** is a polygon that tightly encloses a set of points, offering a more accurate and representative boundary for the fire's extension than a convex hull would. It takes into account the natural shape of the fire, creating more precise curves around the burn areas and active fire zones.

### Ratio Parameter for Concave Hull

The concave hull requires a **ratio** parameter, which determines how tightly the shape fits around the points. A ratio of `0.3` is typically used, as it slightly overestimates the area, which is generally preferred in fire monitoring to ensure safety and a more cautious approach. However, you can adjust this ratio depending on the level of precision you need:
- **Lower ratio (e.g., 0.1)**: Results in a more detailed shape with tighter curves, fitting closely to the actual data points.
- **Higher ratio (e.g., 0.5)**: Produces a more relaxed shape that is less tightly fitted to the points, potentially overestimating the fire's boundary.

### Important Considerations:
- **Too Low of a Ratio**: If the ratio is too low, the concave hull may form multiple disconnected fragments instead of a continuous shape. This happens because the shape tries to fit each point very closely, which can lead to undesirable gaps.
  
- **Too High of a Ratio**: If the ratio is too high, the concave hull will become too inflated, covering a larger area than necessary. This could lead to overestimating the fire's true extent.

By adjusting the ratio appropriately, you can generate a more reliable and meaningful representation of the fire's current extension.

In [ ]:
ratio = 0.3

# Generate the concave hull from BAI burned areas and LWIR hotspots
points = []
with fiona.open(output_filename_combined, 'r') as src:
    for feature in src:
        geom = shape(feature['geometry'])
        if geom.geom_type == 'Polygon':
            points.extend(geom.exterior.coords)
        elif geom.geom_type == 'MultiPolygon':
            for poly in geom:
                points.extend(poly.exterior.coords)

multipoint = MultiPoint(points)

concave_hull_geom = concave_hull(multipoint, ratio=ratio)  # Generate the concave hull

concave_output_filename = "concave_combined.shp"

schema = {'geometry': 'Polygon', 'properties': {}}
with fiona.open(output_filename_combined, 'r') as src:
    with fiona.open(concave_output_filename, 'w', driver='ESRI Shapefile', crs=src.crs, schema=schema) as dst:
        dst.write({'geometry': mapping(concave_hull_geom), 'properties': {}})

print(f"Concave hull saved to {concave_output_filename}")

## Conclusion

To accurately estimate the extension of a fire, it is essential to combine multiple data sources, such as BAI burned areas and LWIR hotspots, along with other relevant products. Each of these datasets provides different insights into the fire's behavior, and their integration allows for a more comprehensive and precise assessment of the fire's current and past extent. Relying on a single source could lead to incomplete or misleading conclusions, making the combination of these products a crucial step in fire monitoring and analysis.
